Here what is important for the division score calculation is:

1. calculate_division_score_naive
2. calculate_division_score_phi
3. calculate_division_score_Yuval

The change of the function is done in:
    id_laws['division_score'] = id_laws['vote_id'].apply(calculate_division_score_naive)

I didn't decide which is better and probably ill show a few to "prove" that the insights that are being repeated in the methods, probably true.

Here i also included the code that wrpes the division functions, maybe it'll be helpful



In [ ]:
def get_stats_by_committee_by_knesset_num(knesset_num):
    import pandas as pd
    import os
    from math import sqrt
    encode_for_hebrew = 'windows-1255'
    path_to_id_laws = r"C:\Elchanan\masters\second_year\Semester_B\data\67978-data-haystack-final-project\data\raw\id_laws_knesset"
    path_to_id_laws_specific_knesset = os.path.join(path_to_id_laws, f'id_laws_{knesset_num}_knesset.csv')
    # Load the data
    id_laws = pd.read_csv(path_to_id_laws_specific_knesset ,encoding=encode_for_hebrew)
    
    path_vote_details = r"C:\Elchanan\masters\second_year\Semester_B\data\67978-data-haystack-final-project\data\raw\votes_details_by_knesset\\"
    path_to_id_laws_specific_knesset = os.path.join(path_vote_details, f'votes_details_{knesset_num}_knesset.csv')
    votes_details = pd.read_csv(path_to_id_laws_specific_knesset, encoding=encode_for_hebrew, low_memory=False)
    
    path_vote_mk_info = r"C:\Elchanan\masters\second_year\Semester_B\data\67978-data-haystack-final-project\data\raw\mk_by_specific_knesset\\"
    path_to_id_laws_specific_knesset = os.path.join(path_vote_mk_info, f'mk_by_knesset_{knesset_num}.csv')
    mk_info = pd.read_csv(path_to_id_laws_specific_knesset, encoding=encode_for_hebrew)
    
    # Merge vote details with coalition/opposition information
    votes_details = votes_details.merge(mk_info[['kmmbr_id', 'is_coalition']], on='kmmbr_id', how='left')
    
    # Function to calculate division score for each vote
    def calculate_division_score_naive(vote_id):
        # Filter vote details for the current vote_id
        vote_details = votes_details[votes_details['vote_id'] == vote_id]
        
        # Exclude cancelled votes (vote_result == 0) and non-participating members (vote_result == 4)
        valid_votes = vote_details[(vote_details['vote_result'] != 0) & (vote_details['vote_result'] != 4)]
        
        # Separate coalition and opposition votes
        coalition_votes = valid_votes[valid_votes['is_coalition'] == 1]
        opposition_votes = valid_votes[valid_votes['is_coalition'] == 0]
        
        # Count the agreements between coalition and opposition
        coalition_in_favor = len(coalition_votes[coalition_votes['vote_result'] == 1])
        coalition_against = len(coalition_votes[coalition_votes['vote_result'] == 2])
        coalition_abstained = len(coalition_votes[coalition_votes['vote_result'] == 3])

        opposition_in_favor = len(opposition_votes[opposition_votes['vote_result'] == 1])
        opposition_against = len(opposition_votes[opposition_votes['vote_result'] == 2])
        opposition_abstained = len(opposition_votes[opposition_votes['vote_result'] == 3])

        agreement_score = coalition_in_favor * opposition_in_favor + coalition_against * opposition_against + coalition_abstained*opposition_abstained
        disagreement_score = coalition_in_favor * (opposition_against + opposition_abstained) + coalition_against * (opposition_in_favor + opposition_abstained) + coalition_abstained * (opposition_in_favor + opposition_against)
        
        division_score =   disagreement_score - agreement_score
        
        # Adjust the score by the total number of participants (i.e., 120 members vs fewer participants)
        participation_factor = 1 / 120
        weighted_division_score = division_score * participation_factor
        
        return weighted_division_score
    
        # Function to calculate division score for each vote
    def calculate_division_score_phi(vote_id):
        # Filter vote details for the current vote_id
        vote_details = votes_details[votes_details['vote_id'] == vote_id]
        
        # Exclude cancelled votes (vote_result == 0) and non-participating members (vote_result == 4)
        valid_votes = vote_details[(vote_details['vote_result'] != 0) & (vote_details['vote_result'] != 4)]
        
        # Separate coalition and opposition votes
        coalition_votes = valid_votes[valid_votes['is_coalition'] == 1]
        opposition_votes = valid_votes[valid_votes['is_coalition'] == 0]
        
        # Count the agreements between coalition and opposition
        coalition_in_favor = len(coalition_votes[coalition_votes['vote_result'] == 1])
        coalition_against = len(coalition_votes[coalition_votes['vote_result'] == 2])
        opposition_in_favor = len(opposition_votes[opposition_votes['vote_result'] == 1])
        opposition_against = len(opposition_votes[opposition_votes['vote_result'] == 2])
        
        
        # Calculate the total number of coalition and opposition members who participated in the vote
        total_coalition = len(coalition_votes)
        total_opposition = len(opposition_votes)
        total_in_favor = coalition_in_favor + opposition_in_favor
        total_against = coalition_against + opposition_against
        total_participation = total_coalition + total_opposition

        # Calculate the level of agreement/disagreement between coalition and opposition   
        
        # Division score: the more agreement, the higher the score
        phi_c_o =  abs((coalition_in_favor*opposition_against-coalition_against*opposition_in_favor)/sqrt(total_in_favor*total_against*total_coalition*total_opposition + 1))

        # Adjust the score by the total number of participants (i.e., 120 members vs fewer participants)
        participation_factor = total_participation / 120
        weighted_phi_c_o = phi_c_o * participation_factor
        
        return weighted_phi_c_o
    
            # Function to calculate division score for each vote
    def calculate_division_score_Yuval(vote_id):
        # Filter vote details for the current vote_id
        vote_details = votes_details[votes_details['vote_id'] == vote_id]
        
        # Exclude cancelled votes (vote_result == 0) and non-participating members (vote_result == 4)
        valid_votes = vote_details[(vote_details['vote_result'] != 0) & (vote_details['vote_result'] != 4)]
        
        # Separate coalition and opposition votes
        coalition_votes = valid_votes[valid_votes['is_coalition'] == 1]
        opposition_votes = valid_votes[valid_votes['is_coalition'] == 0]
        
        # Count the agreements between coalition and opposition
        coalition_in_favor = len(coalition_votes[coalition_votes['vote_result'] == 1])
        coalition_against = len(coalition_votes[coalition_votes['vote_result'] == 2])
        opposition_in_favor = len(opposition_votes[opposition_votes['vote_result'] == 1])
        opposition_against = len(opposition_votes[opposition_votes['vote_result'] == 2])
        
        
        # Calculate the total number of coalition and opposition members who participated in the vote
        total_coalition = len(coalition_votes) + 1
        total_opposition = len(opposition_votes) + 1
        total_in_favor = coalition_in_favor + opposition_in_favor
        total_against = coalition_against + opposition_against
        total_participation = total_coalition + total_opposition
        
        # Division score: the more agreement, the higher the score
        yuv_score =   (opposition_against/total_opposition-0.5)*(coalition_against/total_coalition-0.5)*-4

        # Adjust the score by the total number of participants (i.e., 120 members vs fewer participants)
        participation_factor = total_participation / 120
        weighted_phi_c_o = yuv_score * participation_factor
        
        return weighted_phi_c_o
    
    # Apply the function to all votes in id_laws
    # id_laws['division_score'] = id_laws['vote_id'].apply(calculate_division_score_naive)
    
    id_laws['division_score'] = id_laws['vote_id'].apply(calculate_division_score_naive)
    
    # Remove rows where CommitteeID is empty or NaN
    filtered_id_laws_df = id_laws[id_laws['CommitteeID'].notna() & (id_laws['CommitteeID'] != '')]
    
    agg_stats_all = filtered_id_laws_df['division_score'].agg(['mean', 'std', 'median']).reset_index()
    # Count occurrences of each CommitteeID
    committee_counts = filtered_id_laws_df['CommitteeID'].value_counts()
    minimum_occurrences = 10
    # Filter out CommitteeID with fewer than 10 occurrences
    filtered_id_laws_df_sorted_more_10 = filtered_id_laws_df[filtered_id_laws_df['CommitteeID'].isin(committee_counts[committee_counts >= minimum_occurrences].index)].sort_values(by='CommitteeID')
    # filtered_id_laws_df_sorted_more_10
    
    # Group by CommitteeID and CommitteeName and calculate sum, mean, and std for 'division_score'
    agg_stats = filtered_id_laws_df_sorted_more_10.groupby(['CommitteeID', 'CommitteeName'])['division_score'].agg(['mean', 'std', 'median']).reset_index()
    
    # Extract mean, std, and median from agg_stats_all
    mean_value = agg_stats_all.loc[agg_stats_all['index'] == 'mean', 'division_score'].values[0]
    std_value = agg_stats_all.loc[agg_stats_all['index'] == 'std', 'division_score'].values[0]
    median_value = agg_stats_all.loc[agg_stats_all['index'] == 'median', 'division_score'].values[0]
    
    # Create a DataFrame for the new row
    new_row = pd.DataFrame({
        'CommitteeID': [-1],
        'CommitteeName': ['כלל הועדות'],
        'mean': [mean_value],
        'std': [std_value],
        'median': [median_value]
    })
    
    # Concatenate the new row with agg_stats
    agg_stats = pd.concat([agg_stats, new_row], ignore_index=True)

    return agg_stats

In [ ]:
# knesset_num = 20
stats_for_all_knesset = []


for knesset_num in range(16,25):
    stats_for_all_knesset.append(get_stats_by_committee_by_knesset_num(knesset_num=knesset_num))

Here is the part of code that plot:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
operation_to_plot = 'mean'
fontsize = 22
# Hebrew committee names to plot
committees_of_interest = [
    'ועדת הכספים', 
    'ועדת הכלכלה', 
    'ועדת החוקה, חוק ומשפט', 
    'ועדת העבודה, הרווחה והבריאות',
    'כלל הועדות'
]

# Initialize a dictionary to store mean values for each committee across Knessets
committee_means = {committee: [] for committee in committees_of_interest}
knesset_nums = []

# Iterate through the stats_for_all_knesset list
for idx, df in enumerate(stats_for_all_knesset, start=16):
    # Filter the DataFrame for the committees of interest
    filtered_df = df[df['CommitteeName'].isin(committees_of_interest)]
    
    # Check if the filtered DataFrame has the same number of rows as committees_of_interest
    if len(filtered_df) != len(committees_of_interest):
        print(f"Skipping Knesset {idx} due to insufficient data")
        continue  # Skip this Knesset if there isn't enough data
    
    # Append the Knesset number to the list
    knesset_nums.append(idx)
    
    # Collect mean values for each committee
    for committee in committees_of_interest:
        mean_value = filtered_df.loc[filtered_df['CommitteeName'] == committee, operation_to_plot.lower()].values
        # print(f"for {committee}: {mean_value}")
        committee_means[committee].append(mean_value[0] if len(mean_value) > 0 else np.nan)

# Now plot the data
plt.figure(figsize=(12, 8))

# Define colors for each committee
colors = plt.get_cmap('tab10', len(committees_of_interest))

# Plot mean values for each committee
for i, (committee, means) in enumerate(committee_means.items()):
    plt.plot(knesset_nums, means, label=committee, marker='o', color=colors(i))

# Customize x-axis and y-axis labels
plt.xticks(ticks=knesset_nums, labels=knesset_nums)
plt.xlabel('Knesset Number', fontsize=fontsize)
plt.ylabel(f'{operation_to_plot} of Chi Measurement', fontsize=fontsize)
plt.title(f'{operation_to_plot} of Chi Measurement by Committee Across Knessets', fontsize=fontsize)

# Reverse the committee names for the legend
reversed_committee_names = [committee[::-1] for committee in committees_of_interest]
# Add legend with reversed names
plt.legend(reversed_committee_names, title='Committee Name', loc='upper left', bbox_to_anchor=(1, 1), fontsize=fontsize-4, title_fontsize=fontsize-4)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()
